In [1]:
# Импортируем библиотеки:

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
import os
import re
import json
from bs4 import BeautifulSoup
from fake_useragent import UserAgent 
from tqdm import tqdm
from pprint import pprint

In [2]:
# Увеличиваем число отображаемых строк и столбцов в pandas:

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [3]:
# Задаем дирректорию для работы с файлами:

# При выполнении на kaggle раскомментировать этот блок:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
# PATH_to_file = '/kaggle/input/sf-dst-scoring/'

# При выполнении локально раскомментировать этот блок:
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
PATH_to_file = './data/'

./data/.DS_Store
./data/test.csv
./data/sample_submission.csv


In [4]:
# Загрузим наш тестовый датасет:

test = pd.read_csv(PATH_to_file + 'test.csv')

print('Размер тестового датасета: ', test.shape)
display(test.sample(5))

Размер тестового датасета:  (34686, 32)


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
10882,внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x1/11012616...,коричневый,NaN,Продаю отличный автомобиль. Два комплекта рези...,2.0 LTR,177 N12,"{""engine-proof"":true,""start-stop-function"":tru...",дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,77500,2009,"{""code"":""X1"",""name"":""X1"",""ru_name"":""Х1"",""morph...",X1,20d 2.0d AT (177 л.с.) 4WD,5,1603120845,RUB,2012,1101261609,"{""id"":""5018166"",""name"":""20d"",""nameplate"":""20d""...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,3 или более,5 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен
1475,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/rapid/107...,белый,NaN,Переднее крыло и заднее протерто с левой сторо...,1.6 LTR,90 N12,"{""steel-wheels"":true,""esp"":true,""airbag-driver...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,76078,2012,"{""code"":""RAPID"",""name"":""Rapid"",""ru_name"":""Рапи...",RAPID,1.6 MT (90 л.с.),5,1603231842,RUB,2017,1076854771,"{""id"":""20637133"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,2 владельца,2 года и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен
21019,компактвэн,TOYOTA,https://auto.ru/cars/used/sale/toyota/verso/11...,голубой,NaN,Безопасность: 4. Комфорт: 3. Салон: 2. Защита ...,1.8 LTR,147 N12,"{""airbag-passenger"":true,""condition"":true,""abs...",бензин,https://autoru.naydex.net/imflT9272/520f62I9/2...,138000,2009,"{""code"":""VERSO"",""name"":""Verso"",""ru_name"":""Верс...",VERSO,1.8 MT (147 л.с.),5,1603293951,RUB,2010,1101084231,"{""id"":""20388832"",""displacement"":1798,""engine_t...",COMPACTVAN MECHANICAL 1.8,механическая,JAPANESE,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
26425,хэтчбек 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/golf...,серый,NaN,"Авто во владении давно, собственник я, в птс т...",1.4 LTR,122 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,140000,2012,"{""code"":""GOLF"",""name"":""Golf"",""ru_name"":""Гольф""...",GOLF,1.4 MT (122 л.с.),5,1603225120,RUB,2014,1101110613,"{""id"":""9290828"",""displacement"":1395,""engine_ty...",HATCHBACK_5_DOORS MECHANICAL 1.4,механическая,EUROPEAN,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
7260,седан,VOLVO,https://auto.ru/cars/used/sale/volvo/s60/10988...,чёрный,NaN,Есть незначительные недостатки по кузову,2.4 LTR,140 N12,NaN,бензин,https://autoru.naydex.net/ESooa9149/468b9eoMem...,167000,2004,"{""code"":""S60"",""name"":""S60"",""ru_name"":""S60"",""mo...",S60,2.4 AT (140 л.с.),4,1603239099,RUB,2008,1098861710,"{""id"":""2458055"",""displacement"":2435,""engine_ty...",SEDAN AUTOMATIC 2.4,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [5]:
# В датасете присутствуют названия столбцов на русском языке, а так-же трудно воспринимаемые, переименуем их:

test.columns = ['body_type', 'brand', 'car_url', 'color',
                'complectation', 'description', 'engine_volume',
                'engine_power', 'equipment', 'fuel_type', 'image',
                'mileage', 'model_date', 'model_info', 'model_name',
                'name', 'doors_num', 'parsing_unixtime', 'currency',
                'production_date', 'sell_id', 'super_gen', 'configuration',
                'transmission', 'vendor', 'owners_count', 'ownership',
                'docs', 'drive', 'steering', 'state', 'customs']

In [6]:
# Удаляем ненужные столбцы:

test.drop(columns=['customs', 'currency', 'state'], inplace=True)
test.drop(columns=['sell_id', 'car_url', 'image'], inplace=True)

In [7]:
# Заполняем пробел в столбце docs

test.docs = test.docs.fillna(test.docs.value_counts().index[0])

In [8]:
# Преобразуем owners_count в число, которое содержится в начале строки:

test.owners_count = test.owners_count.str[0].astype('int')

In [9]:
# Удаляем configuration т.к. он дублирует body_type, transmission и engine_volume

test.drop(columns='configuration', inplace=True)

In [10]:
# Потом пригодится для общего датасета

#test.engine_volume = test.engine_volume.apply(lambda x: '0.0' if x == ' LTR' 
#                                              else x.split()[0]).astype('float')

IndentationError: unexpected indent (<ipython-input-10-ceebd8f41d9e>, line 4)

In [11]:
# Если не обращать внимания на иногда попадающиеся "приписки", то столбец  name
# содержит информацию об объеме двигателя, типе коробки передач и мощности.
# Удаляем его

test.drop(columns='name', inplace=True)

In [12]:
# Потом пригодится для общего датасета

# test.engine_power = test.engine_power.apply(lambda x: x.split()[0]).astype('int')



In [13]:
test.drop(columns='model_info', inplace=True)
test.drop(columns='super_gen', inplace=True)

In [14]:
months = test.ownership.str.extract(
    '(\d{1,2})\sмес', expand=False).astype('float')
years = test.ownership.str.extract(
    '(\d{1,2})\s(?:лет|год)', expand=False).astype('float')

test.ownership = (years*12 + months)

In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   body_type         34686 non-null  object 
 1   brand             34686 non-null  object 
 2   color             34686 non-null  object 
 3   complectation     6418 non-null   object 
 4   description       34686 non-null  object 
 5   engine_volume     34686 non-null  object 
 6   engine_power      34686 non-null  object 
 7   equipment         24690 non-null  object 
 8   fuel_type         34686 non-null  object 
 9   mileage           34686 non-null  int64  
 10  model_date        34686 non-null  int64  
 11  model_name        34686 non-null  object 
 12  doors_num         34686 non-null  int64  
 13  parsing_unixtime  34686 non-null  int64  
 14  production_date   34686 non-null  int64  
 15  transmission      34686 non-null  object 
 16  vendor            34686 non-null  object

In [19]:
test.docs.unique()

array(['Оригинал', 'Дубликат'], dtype=object)

In [20]:
test.drive.unique()

array(['передний', 'полный', 'задний'], dtype=object)

In [21]:
test.steering.unique()

array(['Левый', 'Правый'], dtype=object)

In [22]:
pages_list = []

for i in range(1, 100):
    for brand in test.brand.unique():
        url = 'https://auto.ru/cars/' + brand + '/used/?page=' + str(i)
        pages_list.append(url)

In [25]:
MyFile = open (PATH_to_file + 'links.txt', 'w') 
#MyList = map (lambda x: x + '\ n', pages_list) 
#MyFile.writelines (MyList) 
#MyFile.close ()

for element in pages_list:
     MyFile.write(element)
     MyFile.write('\n')
MyFile.close()

In [27]:
with open(PATH_to_file + 'links.txt', 'w') as f:
    for link in pages_list:
        f.write(str(link) +"\n")

In [29]:
page_list_2 = []

with open(PATH_to_file + 'links.txt', 'r') as f:
  for line in f:
    page_list_2.append(str(line.strip()))

In [43]:
test[test.vendor == test.vendor.unique()[0]].brand.unique()

array(['SKODA', 'AUDI', 'VOLVO', 'BMW', 'MERCEDES', 'VOLKSWAGEN'],
      dtype=object)

In [44]:
test[test.vendor == test.vendor.unique()[1]].brand.unique()

array(['HONDA', 'NISSAN', 'INFINITI', 'TOYOTA', 'LEXUS', 'MITSUBISHI'],
      dtype=object)

In [51]:
more_brands=['citroen', 'datsun', 'fiat', 'land_rover', 'mazda', 'mini', 
 'opel', 'peugeot', 'porsche', 'renault', 'subaru', 'suzuki'] 

In [45]:
test.brand.unique()

array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'],
      dtype=object)

In [47]:
brands = []

In [49]:
for i in range(len(test.brand.unique())):
    brands.append((test.brand.unique()[i]).lower())

In [53]:
brands.extend(more_brands)

In [54]:
brands

['skoda',
 'audi',
 'honda',
 'volvo',
 'bmw',
 'nissan',
 'infiniti',
 'mercedes',
 'toyota',
 'lexus',
 'volkswagen',
 'mitsubishi',
 'citroen',
 'datsun',
 'fiat',
 'land_rover',
 'mazda',
 'mini',
 'opel',
 'peugeot',
 'porsche',
 'renault',
 'subaru',
 'suzuki']

In [122]:
url = 'https://auto.ru/cars/audi/used/'
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
response.encoding = 'utf-8'
page = BeautifulSoup(response.text, 'html.parser')

In [123]:
json_data = json.loads(page.find('script', type="application/ld+json").string)

In [124]:
json_data

{'@context': 'https://schema.org',
 '@type': 'Car',
 'aggregateRating': {'@type': 'AggregateRating',
  'ratingValue': '4.6247653106839515',
  'bestRating': '5',
  'reviewCount': '2279'},
 'image': '//avatars.mds.yandex.net/get-verba/3587101/2a00000179af710f40b85f9c3aedb68201fd/logo',
 'name': 'AUDI ',
 'offers': {'@type': 'AggregateOffer',
  'highPrice': 18540000,
  'lowPrice': 30000,
  'priceCurrency': 'RUR',
  'offerCount': 2696,
  'offers': [{'@type': 'Offer',
    'url': 'https://auto.ru/cars/used/sale/audi/q3/1105160799-752504bc/',
    'name': 'Audi Q3 I (8U) Рестайлинг',
    'price': 2237400,
    'priceCurrency': 'RUR',
    'availability': 'https://schema.org/InStock',
    'priceValidUntil': '2021-09-13'},
   {'@type': 'Offer',
    'url': 'https://auto.ru/cars/used/sale/audi/r8/1104116209-c49956aa/',
    'name': 'Audi R8 V8 I (Typ 42)',
    'price': 2888400,
    'priceCurrency': 'RUR',
    'availability': 'https://schema.org/InStock',
    'priceValidUntil': '2021-09-13'},
   {'@ty

In [ ]:
{"@context":"https://schema.org","@type":"Car","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.6247653106839515","bestRating":"5","reviewCount":"2279"},"image":"//avatars.mds.yandex.net/get-verba/3587101/2a00000179af710f40b85f9c3aedb68201fd/logo","name":"AUDI ","offers":{"@type":"AggregateOffer","highPrice":18540000,"lowPrice":25000,"priceCurrency":"RUR","offerCount":8106,

In [93]:
container = page.find('div', class_="ListingSearchSave__count")
#txt = container.find('span', class_="ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination-module__pages").text

In [94]:
container.text

'2\xa0696\xa0объявлений'

In [78]:
url = 'https://auto.ru/cars/audi/used/'
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
response.encoding = 'utf-8'
page = BeautifulSoup(response.text, 'html.parser')

In [71]:
page.find('div', class_='ListingPopularMMM PageListing__popularMMM')

<div class="ListingPopularMMM PageListing__popularMMM" id="popularMMM"><div class="ListingPopularMMM__items"><div class="ListingPopularMMM__column"><div class="ListingPopularMMM__item"><a class="Link ListingPopularMMM__itemName" href="https://auto.ru/korolev/cars/audi/used/">Audi</a><div class="ListingPopularMMM__itemCount">2697</div></div><div class="ListingPopularMMM__item"><a class="Link ListingPopularMMM__itemName" href="https://auto.ru/korolev/cars/bmw/used/">BMW</a><div class="ListingPopularMMM__itemCount">4604</div></div><div class="ListingPopularMMM__item"><a class="Link ListingPopularMMM__itemName" href="https://auto.ru/korolev/cars/ford/used/">Ford</a><div class="ListingPopularMMM__itemCount">3263</div></div></div><div class="ListingPopularMMM__column"><div class="ListingPopularMMM__item"><a class="Link ListingPopularMMM__itemName" href="https://auto.ru/korolev/cars/hyundai/used/">Hyundai</a><div class="ListingPopularMMM__itemCount">3992</div></div><div class="ListingPopularM